# Make clean data

In [114]:
import pandas as pd

df = []
#bubbleList = []
#countingList = []
#mergeList = []
#quickList = []
#readCSVList = []
pcList = ["6", "16", "21", "23"]

# for i in range(len(pcList)):
#     bubbleList.append(pd.read_csv("Results"+pcList[i]+"/bubbleSort-PC"+pcList[i]+".csv"))
#     bubbleList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

# for i in range(len(pcList)):
#     countingList.append(pd.read_csv("Results"+pcList[i]+"/countingSort-PC"+pcList[i]+".csv"))
#     countingList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]
    
# for i in range(len(pcList)):
#     mergeList.append(pd.read_csv("Results"+pcList[i]+"/mergeSort-PC"+pcList[i]+".csv"))
#     mergeList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

# for i in range(len(pcList)):
#     quickList.append(pd.read_csv("Results"+pcList[i]+"/quickSort-PC"+pcList[i]+".csv"))
#     quickList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

# for i in range(len(pcList)):
#     readCSVList.append(pd.read_csv("Results"+pcList[i]+"/readCSV-PC"+pcList[i]+".csv"))
#     readCSVList[i].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

# mergeList.append(pd.read_csv("Results"+pcList[1]+"/mergeSort-400runs-PC16.csv"))
# mergeList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

#countingList.append(pd.read_csv("results"+pcList[1]+"/CountingSort-400runs-PC16.csv"))
#countingList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

#### SUMMER ANALYSIS

#df.append(pd.read_csv("quickSort-400runs-PC16.csv"))
#df.append(pd.read_csv("readCSV-PC16.csv"))
#df.append(pd.read_csv("bubbleSort-400runs-PC16.csv"))
df.append(pd.read_csv("bubbleSort-branchMispredict-PC16.csv"))
df[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]



In [115]:
print(df)

[                      Test name      PKG (J)      CPU (J) GPU (J)    DRAM (J)  \
0          greaterThan0-10000-1    89.533447    78.013611            1.915344   
1          greaterThan0-10000-1    89.148987    77.656555            1.911316   
2          greaterThan0-10000-1    87.678711    77.169312            1.745117   
3          greaterThan0-10000-1    84.473633    74.263977            1.692444   
4          greaterThan0-10000-1    84.666931    74.415405            1.696960   
..                          ...          ...          ...     ...         ...   
294  greaterThanEquals0-50000-1  5507.321472  4754.264709          123.079224   
295  greaterThanEquals0-50000-1  5435.864746  4686.072449          122.553528   
296  greaterThanEquals0-50000-1  5032.376038  4281.642883          122.986145   
297  greaterThanEquals0-50000-1  5524.121765  4771.065674          123.157410   
298  greaterThanEquals0-50000-1  5021.861328  4271.816162          122.748840   

     Temp (C)  Time (ms)  

In [116]:
def removeNonsense(df):
    df = df[df["PKG (J)"]>=0]
    df = df[df["CPU (J)"]>=0]
    df = df[df["Time (ms)"]>=0]
    return df

In [117]:
def makeinputColumn(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[0])

    df["Input case"] = l

def makeSizecolumn(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[1]))

    df["Size"] = l

In [118]:
def makeinputColumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[1].lower())
    df["Input case"] = l

def makeSizecolumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[-1]))

    df["Size"] = l

In [119]:
def removeOutliers(df, type_, size, col):
    q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
    q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
    IQR = q3 - q1 
    LF = q1 - 1.5 * IQR
    UF = q3 + 1.5 * IQR

    df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
    df2 = df1[df1["Input case"] == type_][df1["Size"]==size][df1[col] >= LF]
    return df2[col]


In [120]:
def CleanDataframe(df, DataOrBase):
    if DataOrBase == "data":
        cases = ["best", "random", "worst"]
    else:
        cases = ["random", "countingrandom", "countingbest", "countingworst", "mergeworst", "quickbest", "reversesorted", "sorted"]

    cleanList = []
    for case in cases:
        if case != "random" and case != "countingrandom":
            for size in [25000,50000,75000,100000,200000,300000,400000,500000,600000,700000,800000,900000,1000000]:
                cleanPortion = pd.DataFrame({"PKG (J)":[]})
                for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                    cleanPortion[col] = removeOutliers(df, case, size, col)
                
                cleanPortion["Input case"] = [case for _ in range(len(cleanPortion))]
                cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]

                cleanList.append(cleanPortion)
        else:
            for i in range(1,11):
                for size in [25000,50000,75000,100000,200000,300000,400000,500000,600000,700000,800000,900000,1000000]:
                    cleanPortion = pd.DataFrame({"PKG (J)":[]})
                    for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                        cleanPortion[col] = removeOutliers(df, case+str(i), size, col)
                
                    cleanPortion["Input case"] = [case+str(i) for _ in range(len(cleanPortion))]
                    cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]
                    cleanList.append(cleanPortion)
    cleanBubble = pd.concat(cleanList)
    return cleanBubble

In [121]:
def prepareDataframe(df, DataOrBase):
    df = removeNonsense(df)

    if DataOrBase == "data":
        makeinputColumn(df)
        makeSizecolumn(df)
    else:
        makeinputColumn_readCSV(df)
        makeSizecolumn_readCSV(df)    

    return df


In [122]:
def CleanCSV (df, DataOrBase, name, pcNO):
    ready = prepareDataframe(df, DataOrBase)
    clean = CleanDataframe(ready, DataOrBase).dropna().reset_index(drop=True)
    print(clean)
    #clean.to_csv("FinalAnalysis\\CleanData\\"+name+"Clean-PC"+pcNO+".csv", sep=",", index=False)
    clean.to_csv(name+"Clean-PC"+pcNO+".csv", sep=",", index=False)

In [123]:
# for i in range(len(bubbleList)):
#     CleanCSV(bubbleList[i], "data","Bubble",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(countingList[i], "data","Counting",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(mergeList[i], "data","Merge",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(quickList[i], "data","Quick",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(readCSVList[i], "base","readCSV",pcList[i])

#### SUMMER ANALYSIS

#CleanCSV(df[0], "data","Quick-400runs",pcList[1])
#CleanCSV(df[0], "base","ReadCSV",pcList[1])
#CleanCSV(df[0], "data","BubbleSort-400runs",pcList[1])
CleanCSV(df[0], "data","BubbleSort-BranchMispredict",pcList[1])


<ipython-input-119-50d3f4887162>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
<ipython-input-119-50d3f4887162>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
<ipython-input-119-50d3f4887162>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]


Empty DataFrame
Columns: [PKG (J), CPU (J), DRAM (J), Time (ms), Temp (C), Input case, Size]
Index: []
